In [ ]:
from dotenv import load_dotenv
from openai import AsyncOpenAI, OpenAI, AsyncAzureOpenAI
from typing import Dict
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool, OpenAIChatCompletionsModel
import asyncio
import os
from pathlib import Path

In [40]:
envpath = "/Users/suresh/Documents/Learnings/Ed-doner chatbot/agents/.env"
load_dotenv(dotenv_path=envpath, override=True)

True

### Sales Pitch Agent

In [ ]:
azure_modelname=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')
azure_client = AsyncAzureOpenAI(
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    azure_endpoint=os.getenv('AZURE_OPENAI_API_ENDPOINT'),
    api_version = os.getenv('AZURE_OPENAI_API_VERSION'))
azure_model= OpenAIChatCompletionsModel(model=azure_modelname, openai_client=azure_client)

In [ ]:
model_name = "llama-2-13b-chat"
ollama_client = AsyncOpenAI(base_url='http://localhost:1234/v1', api_key='ollama')
ollama_model= OpenAIChatCompletionsModel(model=model_name, openai_client=ollama_client)

In [55]:
instructions1 = "You are a sales agent working for agentOne, \
a company that builds games for kids, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for agentOne, \
a company that builds games for kids, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a creative sales agent working for agentOne, \
a company that builds games for kids, powered by AI. \
You write concise, to the point cold emails."

In [56]:
sales_agent1 = Agent(name="Sales Agent", instructions=instructions1, model=ollama_model)
sales_agent2 =  Agent(name="Humorous Agent", instructions=instructions2, model=ollama_model)
sales_agent3  = Agent(name="Creative Agent",instructions=instructions3,model=ollama_model)

In [57]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

In [58]:
tools = [tool1, tool2, tool3]

In [59]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model=ollama_model)
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model=ollama_model)
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [60]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    print("subject:", subject)
    print("html_body:", html_body)
    return {"subject": subject, "message": html_body}

In [61]:
email_tools = [subject_tool, html_tool, send_html_email]

In [62]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model=ollama_model,
    handoff_description="Convert an email to HTML and send it")


handoffs = [emailer_agent]

In [65]:
sales_manager_instructions = """
You are a Sales Manager at agentOne. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model=ollama_model)

message = "Send out a cold sales email addressed to Dear CEO"


result = await Runner.run(sales_manager, message)

In [ ]:
result.final_output